<a href="https://colab.research.google.com/github/ZJCHEN0109/AINLP_BERT/blob/main/Fast_Text_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip3 install opencc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 4.0 MB/s 


In [17]:
# coding: utf-8
"""
Extracted the content in the wiki_database
"""
from gensim.corpora import WikiCorpus


# Load data
wiki_corpus = WikiCorpus('zhwiki-20220801-pages-articles1.xml-p1p187712.bz2', dictionary={})


# Save data
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    print('Start to preprocess.')
    for times, text in enumerate(wiki_corpus.get_texts()):
        f.write(' '.join(text)+'\n')

        if (times+1) % 10000 == 0:
            print(times+1)

Start to preprocess.
10000
20000
30000


In [21]:
# coding: utf-8
"""
Tokenize
"""
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')


# Tokenize
with open('wiki_text_seg.txt', 'w', encoding='utf-8') as new_f:
    with open('MedCareOnlyACode.txt', 'r', encoding='utf-8') as f:
        for data in f:
            data = cc.convert(data)
            data = jieba.cut(data)
            data = [word for word in data if word != ' ']
            data = ' '.join(data)

            new_f.write(data)

In [22]:
# coding: utf-8
from gensim.models import word2vec, fasttext


# Settings
seed = 666
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000


# Train
train_data = word2vec.LineSentence('wiki_text_seg.txt')
model = fasttext.FastText(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words,
)


model.save('fasttext.model')

In [24]:
# coding: utf-8
"""
Test the w2v model
"""
from gensim.models import word2vec


# Load the model
model = word2vec.Word2Vec.load('fasttext.model')


# Test
print(model['藥品交互作用'].shape)

for item in model.most_similar('藥品交互作用'):
    print(item)

model.save('fasttext.model')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app


(100,)
('交互作用', 0.9991402626037598)
('famotidine', 0.9977707266807556)
('Famotidine', 0.9977117776870728)
('theophylline', 0.9976571798324585)
('Quetiapine', 0.9975762367248535)
('Theophylline', 0.9975470900535583)
('quetiapine', 0.9973865747451782)
('theophyllin', 0.997315526008606)
('metformine', 0.9970887899398804)
('建議', 0.9970154762268066)
